This notebook takes processed mat/pickle files and makes a dataset for prediction.
In particular, given a history and prediction horizon, this generates all viable slices of a full trajectory.  This goes through all pickle files and constructs a dataset for k-fold cross validation.

### TODO
* [ ] optional: complete outlier removal in bag_processing (see "Additional Notes").  Need to check time indices.
* [ ] optional: add metadata to tfrecord (e.g. snippet #, timestamp).

In [1]:
import glob
import numpy as np
import scipy.io as sio
import pickle
import matplotlib.pyplot as plt
import pdb
from datetime import datetime
import sklearn.utils as sku
from tqdm import tqdm

from pkl_reader import *
from tfrecord_utils import write_tfrecord, read_tfrecord

Using TensorFlow backend.


In [2]:
''' CONFIG: CONSTANTS FOR EXECUTION '''
num_folds_cv = 5 # k-fold cross validation, number of splits (-1 = only provide full set of data)

prune_start=True          # remove stationary portion of ego's trajectory at the start
prune_end=True            # remove stationary portion of ego's trajectory at the end
min_vel_thresh=0.01       # velocity threshold (m/s) above which ego is considered moving
exclude_collisions=True  # return an empty trajectory if there was a collision

Nhist=5          # number of timesteps of motion history to predict with
Npred=20         # number of timesteps of prediction horizon
Nskip=5          # "stride" for sliding window of snippet selection
dt=0.1           # discretization (s) of full ego trajectory corresponding to N* above
ego_trans = True # whether or not to represent trajectory snippets in the ego frame
                 # if False, use the global map frame for all snippets

In [3]:
save_ext = 'pkl'
file_prefix = '../examples/bags/'
search_str = file_prefix + '*.' + save_ext 
files_to_process = glob.glob(search_str)
print('Found %d files to read: %s' % (len(files_to_process), files_to_process))

Found 57 files to read: ['../examples/bags/parking_p2_t1_e4_2019-11-04-16-34-11.pkl', '../examples/bags/parking_p1_t1_e6_2019-11-04-14-36-57.pkl', '../examples/bags/parking_p2_t1_e5_2019-11-04-16-34-40.pkl', '../examples/bags/parking_p1_t2_e5_2019-11-04-14-43-12.pkl', '../examples/bags/parking_p3_t2_e2_2019-11-26-16-12-23.pkl', '../examples/bags/parking_p1_t1_e1_2019-11-04-14-34-58.pkl', '../examples/bags/parking_p3_t2_e4_2019-11-26-16-12-49.pkl', '../examples/bags/parking_p3_t2_e6_2019-11-26-16-13-37.pkl', '../examples/bags/parking_p1_t2_e2_2019-11-04-14-41-42.pkl', '../examples/bags/parking_p2_t1_e9_2019-11-04-16-35-54.pkl', '../examples/bags/parking_p1_t1_e5_2019-11-04-14-36-41.pkl', '../examples/bags/parking_p2_t2_e3_2019-11-04-16-39-13.pkl', '../examples/bags/parking_p2_t1_e8_2019-11-04-16-35-33.pkl', '../examples/bags/parking_p3_t1_e0_2019-11-26-15-45-03.pkl', '../examples/bags/parking_p3_t2_e7_2019-11-26-16-13-59.pkl', '../examples/bags/parking_p2_t1_e1_2019-11-04-16-33-19.pkl',

In [4]:
num_tfrecords = 0
for file in tqdm(files_to_process):
    if save_ext == 'pkl':
        res_dict = pickle.load(open(file,'rb'))
    else:
        raise NotImplemented('Invalid Extension')
    
    goals = extract_goals(res_dict)
    parking_lot = res_dict['parking_lot']
    ego_dimensions = res_dict['ego_dimensions']
    static_object_list = res_dict['static_object_list']
    
    try:
        ego_trajectory, start_ind, switch_ind, end_ind, goal_ind = \
             extract_full_trajectory(res_dict, goals, prune_start, prune_end, \
                                     min_vel_thresh, exclude_collisions)

        features, scene_images, labels, goal_snpts = \
            get_ego_trajectory_prediction_snippets(ego_trajectory, start_ind, switch_ind, end_ind, goal_ind, \
                                           goals, parking_lot, ego_dimensions, static_object_list, \
                                           Nhist, Npred, Nskip, dt, ego_frame=ego_trans)
        
        '''
        # For visualization/debugging: see the first snippet of data.
        plt.figure(figsize=(10, 10), dpi=160, facecolor='w', edgecolor='k')
        for i in range(Nhist):
            plt.subplot(1, Nhist, i+1)
            plt.imshow(scene_images[0, i, :, :, :])
        plt.tight_layout()
        plt.show()
        '''

        file_location = file_prefix + file.split('/')[-1].split('.')[0] + '.tfrecord'
        write_tfrecord(features, scene_images, labels, goal_snpts, file_location, {})
        
        '''
        # For debugging: see an arbitrary snippet from the generated tfrecord.
        image, feature, label, goal, count = read_tfrecord([file_location])
        
        plt.figure(figsize=(10, 10), dpi=160, facecolor='w', edgecolor='k')
        for i in range(Nhist):
            plt.subplot(1, Nhist, i+1)
            plt.imshow(image.numpy()[0, i, :, :, :])
        plt.tight_layout()
        plt.show()
        '''
        num_tfrecords += 1
                
    except ValueError as e:
        print(file, e)
        
print(num_tfrecords, ' tfrecord files were written.')
    

 37%|███▋      | 21/57 [00:18<00:35,  1.01it/s]

Collisions encountered: 
time 1218.723575582
other_id 868 vehicle.ford.mustang_autopilot
normal_impulse [-301.71484375, -1832.911865234375, 34.343990325927734]
time 1218.823575587
other_id 868 vehicle.ford.mustang_autopilot
normal_impulse [-42.54573059082031, -468.8822937011719, 8.161503791809082]
../examples/bags/parking_p2_t2_e0_2019-11-04-16-36-50.pkl Collision encountered, so skipping this instance.


 63%|██████▎   | 36/57 [00:29<00:14,  1.46it/s]

Collisions encountered: 
time 2292.091427611
other_id 1367 vehicle.chevrolet.impala_autopilot
normal_impulse [2206.912353515625, -4.1921868324279785, -11.82215404510498]
time 2292.091427611
other_id 1367 vehicle.chevrolet.impala_autopilot
normal_impulse [285.27642822265625, -0.5919407606124878, -2.3527565002441406]
time 2292.158094281
other_id 1367 vehicle.chevrolet.impala_autopilot
normal_impulse [337.15869140625, -0.7436597347259521, -3.806938886642456]
time 2292.191427616
other_id 1367 vehicle.chevrolet.impala_autopilot
normal_impulse [344.3897705078125, -0.759609043598175, -3.8885862827301025]
time 2292.224760951
other_id 1367 vehicle.chevrolet.impala_autopilot
normal_impulse [343.02374267578125, -0.756596028804779, -3.873162031173706]
time 2292.291427621
other_id 1367 vehicle.chevrolet.impala_autopilot
normal_impulse [324.8485412597656, -0.7165075540542603, -3.6679418087005615]
time 2292.424760961
other_id 1367 vehicle.chevrolet.impala_autopilot
normal_impulse [292.2283020019531, 

 68%|██████▊   | 39/57 [00:31<00:10,  1.73it/s]

Collisions encountered: 
time 428.591330422
other_id 796 vehicle.nissan.patrol_autopilot
normal_impulse [48.102664947509766, 38.11619567871094, -0.8094014525413513]
time 428.691330427
other_id 796 vehicle.nissan.patrol_autopilot
normal_impulse [46.12480545043945, 36.548954010009766, -0.7761209011077881]
time 428.724663762
other_id 796 vehicle.nissan.patrol_autopilot
normal_impulse [47.58728790283203, 37.70781707763672, -0.800729513168335]
time 428.791330432
other_id 796 vehicle.nissan.patrol_autopilot
normal_impulse [20.486284255981445, 16.233179092407227, -0.3447132706642151]
time 428.791330432
other_id 796 vehicle.nissan.patrol_autopilot
normal_impulse [17.838239669799805, 14.134889602661133, -0.3001558780670166]
time 428.824663767
other_id 796 vehicle.nissan.patrol_autopilot
normal_impulse [38.68595504760742, 30.65446662902832, -0.6509508490562439]
time 428.957997107
other_id 796 vehicle.nissan.patrol_autopilot
normal_impulse [45.07197952270508, 35.71470260620117, -0.758405506610870

100%|██████████| 57/57 [00:47<00:00,  1.19it/s]

54  tfrecord files were written.
